In [1]:
import pandas as pd 
import json
import re
import os
from deep_translator import GoogleTranslator
import time
from langdetect import detect

## Load

In [2]:
'''
Obtain the last json file number
'''
data_dir = os.path.join(os.getcwd(), "temp")

def get_last_saved_batch(temp_dir):
    files = os.listdir(temp_dir)
    json_files = [f for f in files if re.match(r"output_\d+\.json", f)]
    if not json_files:
        return 0  # No files

    # Extract last file number
    numbers = [int(re.search(r"output_(\d+)\.json", f).group(1)) for f in json_files]
    return max(numbers)

last_file_num = get_last_saved_batch(data_dir)

In [3]:
'''
Load the jsons data into a dataframe
'''
data = []
for i in range(last_file_num):
    filename = os.path.join(data_dir,f"output_{i+1}.json")
    with open(filename, "r", encoding="utf-8") as f:
        try:
            content = json.load(f)
            data.extend(content)
        except json.JSONDecodeError:
            print(f"Cannot open {filename}")
print("Data loaded successfuly")

df_json = pd.DataFrame(data)

Data loaded successfuly


In [4]:
df_json.head(2)

,title,author,isbn,description,subjects,num_pages,genres,avgRating
0,Yo Me He Llevado Tu Queso,Robert Bristow,8466610774,None,[Self-actualization (Psychology)],198.0,[Self-actualization (Psychology)],4.5
1,"Reyes Malditos I, Los. El Rey de Hierro",Maurice Druon,8466612041,La terrible maldicion de los templarios.B!Todo...,[Fiction],296.0,[Fiction],4.0


In [5]:
print(df_json.shape)

(147000, 8)


In [6]:
'''
Take the interested cols and drop the others
'''
important_cols = ["isbn", "description", "num_pages", "genres", "avgRating"]
non_cols = [col for col in df_json.columns.tolist() if col not in important_cols]
df_json = df_json.drop(non_cols, axis = 1)

In [7]:
df_json.head(2)

,isbn,description,num_pages,genres,avgRating
0,8466610774,None,198.0,[Self-actualization (Psychology)],4.5
1,8466612041,La terrible maldicion de los templarios.B!Todo...,296.0,[Fiction],4.0


In [8]:
'''
Load the original data
'''
column_names = ["title", "author", "year", "rating", "isbn", "url"]
raw_df = pd.read_csv("library_for_scrapping.csv", sep=",", quotechar='"',on_bad_lines="skip",encoding='latin-1', names = column_names)

In [9]:
raw_df.head(2)

,title,author,year,rating,isbn,url
0,Yo Me He Llevado Tu Queso,Robert Bristow,2003,10.0000,8466610774,http://images.amazon.com/images/P/8466610774.0...
1,"Reyes Malditos I, Los. El Rey de Hierro",Maurice Druon,2003,10.0000,8466612041,http://images.amazon.com/images/P/8466612041.0...


In [10]:
raw_df.shape

(147000, 6)

In [11]:
'''
Merge both dataframes
'''
df_merged = pd.merge(raw_df, df_json, on="isbn", how="inner")

In [12]:
df_merged.head(2)

,title,author,year,rating,isbn,url,description,num_pages,genres,avgRating
0,Yo Me He Llevado Tu Queso,Robert Bristow,2003,10.0000,8466610774,http://images.amazon.com/images/P/8466610774.0...,None,198.0,[Self-actualization (Psychology)],4.5
1,"Reyes Malditos I, Los. El Rey de Hierro",Maurice Druon,2003,10.0000,8466612041,http://images.amazon.com/images/P/8466612041.0...,La terrible maldicion de los templarios.B!Todo...,296.0,[Fiction],4.0


## Transform

In [43]:
# Checkpoint
df_sample = df_merged.copy()

In [44]:
'''
Clean dataframe
'''

df_sample['genres'] = df_sample['genres'].apply(lambda x: ', '.join(x))
df_sample = df_sample[df_sample["num_pages"] > 0]
df_sample = df_sample[df_sample["genres"] != ""]
df_sample = df_sample[df_sample["description"].notna()]

In [45]:
df_sample = df_sample.reset_index(drop=True)
df_sample.shape

(73263, 10)

In [46]:
df_sample.head(2)

,title,author,year,rating,isbn,url,description,num_pages,genres,avgRating
0,"Reyes Malditos I, Los. El Rey de Hierro",Maurice Druon,2003,10.0000,8466612041,http://images.amazon.com/images/P/8466612041.0...,La terrible maldicion de los templarios.B!Todo...,296.0,Fiction,4.0
1,El Ocho,Katherine Neville,2004,10.0000,8466616160,http://images.amazon.com/images/P/8466616160.0...,Las valiosas piezas del juego de ajedrez de Mo...,860.0,Fiction,4.0


In [47]:
'''
Verify type of information in the current df
'''
types = {}
cols = df_sample.columns.tolist()
for i in range(df_sample.shape[1]):
    types[cols[i]] = df_sample[cols[i]].map(type).value_counts()
print(types)

{'title': <class 'str'>    73263
Name: title, dtype: int64, 'author': <class 'str'>    73263
Name: author, dtype: int64, 'year': <class 'str'>    73263
Name: year, dtype: int64, 'rating': <class 'str'>    73263
Name: rating, dtype: int64, 'isbn': <class 'str'>    73263
Name: isbn, dtype: int64, 'url': <class 'str'>    73263
Name: url, dtype: int64, 'description': <class 'str'>    73263
Name: description, dtype: int64, 'num_pages': <class 'float'>    73263
Name: num_pages, dtype: int64, 'genres': <class 'str'>    73263
Name: genres, dtype: int64, 'avgRating': <class 'float'>    73263
Name: avgRating, dtype: int64}


In [ ]:
'''
There are some mistakes in the year formating
'''

def is_numeric(val):
    try:
        if int(val) or float(val)
            return True
    except:
        return False

#No string values
df_sample['is_num_y'] = df_sample['year'].apply(is_numeric)
df_sample = df_sample[df_sample['is_num_y']]
#No numeric values
df_sample['genre_num'] = df_sample['genres'].apply(is_float)
df_sample = df_sample[~df_sample['genre_num']]

In [48]:
'''
Convert rating into float and combine it with avgRating from scrapping
'''
def is_float(val):
    try:
        float(val)
        return True
    except:
        return False

# Rating as float
df_sample['rating_is_float'] = df_sample['rating'].apply(is_float)
df_sample = df_sample[df_sample['rating_is_float']]

df_sample['rating'] = df_sample['rating'].apply(lambda x: float(x)) / 2
df_sample['avgRating'] = df_sample['avgRating'].combine_first(df_sample['rating'])

In [54]:
'''
For RAG chatbot I will use only english descriptions and genres, therefore we need to translate the descriptions 
that are not in english
'''
def detect_language_safe(x):
    if isinstance(x, str) and x.strip() != "":
        try:
            return detect(x)
        except:
            return "unknown"
    return "unknown"

def translate_to_english(text, lang):
    try:
        if lang != "en":
            return GoogleTranslator(source='auto', target='en').translate(text)
        else:
            return text
    except:
        return text


In [ ]:
#Descriptions
df_sample["lang"] = df_sample["description"].apply(detect_language_safe)
df_sample = df_sample[df_sample['lang'] != "unknown"]


In [62]:
#Gneres
df_sample["lang_genres"] = df_sample["genres"].apply(detect_language_safe)
df_sample = df_sample[df_sample['lang_genres'] != "unknown"]

In [71]:
df_sample.head(2)

,title,author,year,rating,isbn,url,description,num_pages,genres,avgRating,rating_is_float,lang,lang_genres
0,"Reyes Malditos I, Los. El Rey de Hierro",Maurice Druon,2003,5.0,8466612041,http://images.amazon.com/images/P/8466612041.0...,The terrible cursion of the Templars. B! All d...,296.0,Fiction,4.0,True,es,en
1,El Ocho,Katherine Neville,2004,5.0,8466616160,http://images.amazon.com/images/P/8466616160.0...,The valuable pieces of the Montglane chess gam...,860.0,Fiction,4.0,True,es,en


In [115]:
#checkpoint2 = df_sample.copy()
df_sample = checkpoint2.copy()

In [58]:
'''
Translate descriptions and genres != en
'''
from tqdm import tqdm
tqdm.pandas()

#Descriptions
df_sample["description"] = df_sample.progress_apply(
    lambda row: translate_to_english(row["description"], row["lang"]),
    axis=1
)

100%|██████████| 73257/73257 [1:07:37<00:00, 18.06it/s]  


In [72]:
#genres
df_sample["genres"] = df_sample.progress_apply(
    lambda row: translate_to_english(row["genres"], row["lang_genres"]),
    axis=1
)

100%|██████████| 73257/73257 [2:03:04<00:00,  9.92it/s]  


In [121]:
'''
Last verification and filters
'''
df_sample = df_sample[df_sample['description'].str.len() > 35]
df_sample["avgRating"] = df_sample["avgRating"].astype(float).round(1)
df_sample['num_pages'] = df_sample['num_pages'].astype(float).round(0).astype(int)


In [124]:
RAG_df = df_sample.drop(["rating", "rating_is_float", "url", "lang", "lang_genres", "is_num_y", "genre_num"], axis = 1)
new_cols = {"num_pages" : "pages", "avgRating" : "rating"}
RAG_df.rename(columns=new_cols, inplace=True)

In [126]:
RAG_df.head(2)

,title,author,year,isbn,description,pages,genres,rating
0,"Reyes Malditos I, Los. El Rey de Hierro",Maurice Druon,2003,8466612041,The terrible cursion of the Templars. B! All d...,296,Fiction,4.0
1,El Ocho,Katherine Neville,2004,8466616160,The valuable pieces of the Montglane chess gam...,860,Fiction,4.0


In [128]:
RAG_df.to_csv("RAG_df.csv", index=False, encoding="utf-8")